# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,8.34,72,82,4.02,NZ,1726440073
1,1,kodiak,57.7900,-152.4072,15.73,41,20,6.69,US,1726440178
2,2,vilyuchinsk,52.9306,158.4028,9.15,56,74,7.84,RU,1726440179
3,3,keflavik,64.0049,-22.5624,5.24,60,0,4.12,IS,1726440180
4,4,adamstown,-25.0660,-130.1015,20.27,81,100,6.85,PN,1726440181


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

city_map = city_data_df.hvplot.points(
                                        "Lng",                                     
                                        "Lat",
                                        geo = True,
                                        tiles = "OSM",
                                        color = "City",
                                        alpha = 0.6,
                                        frame_width = 650,
                                        frame_height = 500
                                     )
# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21)
                                       & (city_data_df["Cloudiness"] == 0)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()
# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
114,114,cromwell,41.5951,-72.6454,22.48,70,0,2.24,US,1726440305
213,213,jamestown,42.0970,-79.2353,24.80,43,0,3.09,US,1726440417
255,255,diyarbakir,37.9158,40.2189,22.99,40,0,1.09,TR,1726440467
257,257,ingeniero guillermo n. juarez,-23.9000,-61.8500,21.60,27,0,3.22,AR,1726440469
278,278,zhob,31.3411,69.4481,23.04,53,0,1.44,PK,1726440492
299,299,al jawf,29.5000,38.7500,24.00,51,0,4.30,SA,1726440516
314,314,chibougamau,49.9168,-74.3659,23.79,60,0,4.12,CA,1726440534
337,337,navalmoral de la mata,39.8916,-5.5406,21.94,22,0,0.97,ES,1726440562
345,345,vernon,50.2581,-119.2691,21.74,33,0,2.68,CA,1726439991
346,346,waterloo,43.4668,-80.5164,26.27,44,0,4.12,CA,1726440573


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
114,cromwell,US,41.5951,-72.6454,70,
213,jamestown,US,42.0970,-79.2353,43,
255,diyarbakir,TR,37.9158,40.2189,40,
257,ingeniero guillermo n. juarez,AR,-23.9000,-61.8500,27,
278,zhob,PK,31.3411,69.4481,53,
299,al jawf,SA,29.5000,38.7500,51,
314,chibougamau,CA,49.9168,-74.3659,60,
337,navalmoral de la mata,ES,39.8916,-5.5406,22,
345,vernon,CA,50.2581,-119.2691,33,
346,waterloo,CA,43.4668,-80.5164,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = { "categories": "accommodation.hotel",
           "apiKey": {geoapify_key}   
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cromwell - nearest hotel: SpringHill Suites Hartford Cromwell
jamestown - nearest hotel: DoubleTree Jamestown
diyarbakir - nearest hotel: Prestige otel ofis
ingeniero guillermo n. juarez - nearest hotel: Hotel El Indiano
zhob - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
chibougamau - nearest hotel: No hotel found
navalmoral de la mata - nearest hotel: Hotel Moya
vernon - nearest hotel: Okanagan Royal Park Inn
waterloo - nearest hotel: Delta Waterloo
barcelona - nearest hotel: Cram
shiraz - nearest hotel: هتل نصیرالملک
mutuali - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
114,cromwell,US,41.5951,-72.6454,70,SpringHill Suites Hartford Cromwell
213,jamestown,US,42.0970,-79.2353,43,DoubleTree Jamestown
255,diyarbakir,TR,37.9158,40.2189,40,Prestige otel ofis
257,ingeniero guillermo n. juarez,AR,-23.9000,-61.8500,27,Hotel El Indiano
278,zhob,PK,31.3411,69.4481,53,No hotel found
299,al jawf,SA,29.5000,38.7500,51,No hotel found
314,chibougamau,CA,49.9168,-74.3659,60,No hotel found
337,navalmoral de la mata,ES,39.8916,-5.5406,22,Hotel Moya
345,vernon,CA,50.2581,-119.2691,33,Okanagan Royal Park Inn
346,waterloo,CA,43.4668,-80.5164,44,Delta Waterloo


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng",
                                   "Lat",
                                   geo = True,
                                   color = "City",
                                   alpha = 0.6,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"],
                                   frame_width= 700,
                                   frame_height= 500
                                  )

# Display the map
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)